# 介绍网络爬取 – Web Scraping

本章我们将介绍如何获取网页数据，并自动爬取，如使用 [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

## 一、HTML 与 DOM

我们将抓取(部分)用HTML编写并在DOM中表示的web页面。DOM代表文档对象模型，HTML代表“超文本标记语言”。25年前，这曾经是对HTML实际功能的一种有意义的描述:它有链接(超文本)，而且它是一种标记语言。然而，最新版本的HTML, HTML5标准，做了更多的事情:图形、音频、视频等等。因此，很容易把HTML看作是“web浏览器知道如何解释的任何东西”，而不考虑实际的术语。

### 1. 元素Elements

HTML的重要之处在于标记是由元素表示的。HTML元素是内容的一部分，由一对相同名称的标记包围。像这样:

```html
<strong>这是HTML元素.</strong>
```

在这个元素中，strong是标记的名称;打开标签是' <strong> '，匹配的结束标签是' </strong> '。你应该这样理解，文本“这是一个HTML元素”应该是“强的”，即。，通常这将是粗体文本。
HTML元素通常可以做嵌套:

```html
<strong>这是加粗并且是 <u>下划线且加粗.</u></strong>
```

除了名称之外，开始标记还可以包含关于元素的额外信息。这些被称为属性:

```html
<a href='http://www.google.com'>Google主页的链接</a>
```
在本例中，我们使用了代表“锚”的“a”元素，但现在几乎普遍用作“链接”。属性“href”表示“HTML引用”，这实际上对更改是有意义的。赋予每个属性的意义在元素之间变化。
对于我们的目的，重要的属性是“id”和“class”。id属性为属性提供一个惟一的标识符，然后可以使用该标识符以编程方式访问元素。可以将其视为通过全局变量访问元素。 
类在概念上是相似的，但它的意图是应用于整个元素的“class”。



HTML页面需要一些样板（boilerplate）文件。下面一个最小的页面HTML: 

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title></title>
</head>
<body>
世界你好！怎么样?
</body>
</html>
``` 

 `<head>` 包含元信息，如网站的标题, `<body>` 包含实际数据. 

### 层次结构Hierarchy

HTML中的数据通常是分层次结构组织的: 

```html
<body>
  <article>
    <span class="date">Published: 2016-08-25</span>
    <span class="author">Led Zeppelin</span>
    <h1>Ramble On</h1>
    <div class="content">
    到处都是落叶，我该走了。
    谢谢你，让我在这儿过得很愉快。
    但现在我该走了。秋天的月亮为我照亮道路。
    因为现在我闻到了雨的味道，伴随着疼痛，它朝我走来。. 
    <div>
  </article>
  <article>
    <span class="date">Published: 2016-08-23</span>
    <span class="author">Radiohead</span>
    <h1>Burn the Witch</h1>
    <div class="content">
    躲在阴影里
    在绞刑架前欢呼
    这是一个围捕
    这是低空飞行引起的恐慌
    在点唱机上唱首歌
    烧女巫
    烧女巫
    我们知道你住哪儿
    <div>
  </article>
</body>
```

这里，歌曲的标题由三层嵌套: `body > article > h1`.

### 2. 表格Tables

数据也通常存为HTML表格（HTML tables）. `<tr>` 指一行 (table row), `<th>` 和 `<td>` 用于区别单元格，或者头单元格 (`<th>`) 或普通单元格 (`<td>`). 

```html
<table>
    <tr>
        <th></th>
        <th>The Beatles</th>
        <th>Led Zeppelin</th>
    </tr>
    <tr>
        <td># Band Members</td>
        <td>4</td>
        <td>4</td>
    </tr>
</table>
```

### 3. 文档对象模型DOM

正如我们在上面看到的，标记文档看起来很像一棵树:它有根，HTML元素，元素可以有包含元素本身的子元素。
虽然HTML是标记文档的文本表示，但是DOM是它的编程接口。DOM还表示页面呈现时的状态，这(现在)并不意味着存在与之完全对应的底层HTML文档。相反，DOM是用JavaScript等动态生成的。
在这个类中，我们将使用“DOM”表示由web浏览器创建的树来表示文档.

#### 检查浏览器中的 DOM

在进行抓取时，最重要的习惯可能是使用开发人员工具调查页面的源代码。在本例中，我们将通过单击菜单栏查看元素树:视图→开发人员→开发人员工具。
你也可以右键点击网页的任何部分，选择“Inspect Element”。请注意，DOM中的内容和源中的内容之间可能存在很大的差异。
看下[这个html网页](lyrics.html)的DOM. 接下来，我们将从这个网页爬取数据! 

## 二、使用BeautifulSoup爬取数据Scraping

[BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) 是一个用于从html文档中计算提取数据的Python库设计。它支持在DOM中导航并精确地返回所需的数据元素.

让我们使用[lyrics.html](lyrics.html) 文件开始简单的例子.

In [ ]:
from bs4 import BeautifulSoup

# 我们告诉BeautifulSoup并告诉它使用哪个解析器
song_soup = BeautifulSoup(open("lyrics.html"), "html.parser")
# 输出与html文件完全对应
song_soup

In [ ]:
# 有时很难读懂，所以我们可以格式化它
print(song_soup.prettify())

我们可以靠标签（tags）访问内容:

In [ ]:
# 获取标题（title）标签，并且从标签获取文本内容
song_soup.title

In [ ]:
song_soup.title.string

In [ ]:
song_soup.title.text

直接访问一个元素对于标签的第一次出现是有效的，我们不会得到其他的. 接着，我们获取元素的文本内容。

In [ ]:
song_soup.div

In [ ]:
print(song_soup.div.string)

我们可以使用属性attributes来查找特定的元素:

In [ ]:
song_soup.find(id="zep")

 我们也可以只获取文本内容，而不是html标记: 

In [ ]:
text = song_soup.find(id="zep").get_text()
print(text)

我们还可以使用find_all来获取标签（tag）的所有实例:

In [ ]:
#返回beautiful soup元素的列表
h1s = song_soup.find_all("h1")
h1s

In [ ]:
h1s[0]

很容易从这个变量里获取文本内容:

In [ ]:
string_h1s = [tag.get_text() for tag in h1s]
string_h1s

由于' find_all '非常常用，所以可以通过直接调用对象来使用快捷方式:

In [ ]:
song_soup("div")

我么可以在返回对象中直接定位元素:

In [ ]:
song_soup("div")[1]

或者在其上进行迭代操作:

In [ ]:
for p in song_soup.find_all("div"):
    print("---")
    print(p)

###CSS选择器- CSS Selectors

我们也可以使用CSS选择器。CSS选择器应用于元素、类和id等。
下面是一个如何使用CSS对不同元素进行样式设置的示例. 


```CSS
/* Element选择器 */
article {
  color: FireBrick;
}

/* ID选择器 */
#myID {
  color: Tomato;
}

/* Class选择器 */
.myClass {
  color: Aquamarine;
}

/* 子选择器。仅直接子匹配 */
p > b {
  color: SteelBlue;
}

/* Descendant（后代）选择器。每当在一个div中嵌套一个b时，它就会匹配 */
div b {
  color: green;
}

```

[这个例子](https://jsfiddle.net/gxhqv26m/1/) 带有所有重要的选择器.

让我们在Python中试试！


In [ ]:
# 选择类".content"的所有元素
song_soup.select(".content")

In [ ]:
# 选择树中id radio下面的所有div
song_soup.select("#radio div")

好了，现在我们知道如何从网站中提取信息了。现在让我们看一个完整的例子。

## 三、获取一个网站Website

下载网站既简单又高效。事实证明，当你大量收集数据时，你会给服务器带来相当高的负载。因此，网站管理员通常会在他们的网站上公布他们所允许的抓取类型。你应该看看网站上的服务条款和`robots.txt`的一个域之前爬取过度。服务条款通常很宽泛，所以搜索“抓取”（scraping）或“爬行”（crawling）是一个好主意。

让我们看看 [Google Scholar's robots.txt](https://scholar.google.com/robots.txt):

```
User-agent: *
Disallow: /search
Allow: /search/about
Disallow: /sdch
Disallow: /groups
Disallow: /index.html?
Disallow: /?
Allow: /?hl=
...
Disallow: /scholar
Disallow: /citations?
...
```

在这里，它指定您不允许爬取许多页面。' /scholar '子目录尤其麻烦，因为它禁止您动态地生成查询。
这也是常见的网站要求你延迟crawiling:
```
Crawl-delay: 30 
Request-rate: 1/30 
```

你应该尊重这些限制。现在，没有人可以阻止你通过爬虫程序运行一个请求，但是像谷歌scholar这样的网站会很快地阻止你，如果你在短时间内请求了很多页面。
动态爬行的另一种策略是下载网站的本地副本并进行爬取。这可以确保您每个页面只访问站点一次。这是一个很好的工具 [wget](https://www.gnu.org/software/wget/). 

### 例子: Utah大学课程注册

我们将建立一个今秋在美国开设的课程数据集，并查看注册人数。我们将使用这里列出的目录:  
https://student.apps.utah.edu/uofu/stu/ClassSchedules/main/1184/

Utah大学似乎不关心是否或者我们如何爬取网站， whether/how we crawl the websites, the [fineprint](https://www.utah.edu/disclaimer/) doesn't mentione it and there is no `robots.txt`: http://www.utah.edu/robots.txt

We'll use the [`urllib.request`](https://docs.python.org/3.0/library/urllib.request.html) library to retreive the websites.

In [ ]:
import urllib.request
import ssl
from bs4 import BeautifulSoup
ssl._create_default_https_context = ssl._create_unverified_context
url = "https://student.apps.utah.edu/uofu/stu/ClassSchedules/main/1184/"
#url = "https://jwc.cueb.edu.cn/jgsz/"
# here we actually access the website
with urllib.request.urlopen(url) as response:
    html = response.read()
    #html = html.decode('utf-8')


# save the file
#print(html)
with open('class_schedule.html', 'w') as new_file:
#with open('cueb_jwc.html', 'w') as new_file:
    new_file.write(str(html))

# here it's already a local operation
soup = BeautifulSoup(html, 'html.parser')


让我们看看网页的头5000行: 

In [ ]:
soup

In [ ]:
print(soup.prettify()[0:5000])

我们想从这个页面得到的是特定学科课程列表的链接。
虽然您可以在python的文本输出中找到这一点，但是在浏览器的内置检查器中找到相关部分要容易得多。这里，我们突出了其中一个课程麻醉学（(Anesthesiology): 


这是HTML相关片段: 

```html
<ul class="subject-list">
  <li class="row"><a class="col-sm-4" href="class_list.html?subject=ANES">ANES</a><span class="col-md-10">Anesthesiology</span></li>
</ul>
```

我们可以基于 "subject-list" 类属性获取. 

让我们构建一个主题缩写的字典，包含完整的主题名称和相关课程的链接。对于CS，它应该是这样的: 

```
CS: (Computer Science, https://student.apps.utah.edu/uofu/stu/ClassSchedules/main/1184/class_list.html?subject=CS)
```

In [ ]:
subjects = {}
#print(soup)
for subject in soup.find_all(class_="subject-list"):
    # 这个url是相对的.
    # 我们可以通过从"a"标签的href属性中检索链接来获得尾部
    print(subject)
    link_tail = subject.find("a").get("href")
    # 连接URL基础与链接的尾部
    link = url + link_tail
    # 主题shortname内嵌在<a>标签
    subject_short = subject.find("a").get_text()
    # 主题名内嵌在span中 
    subject_long = subject.span.get_text()
    # 写入到字典中
    subjects[subject_short] = (subject_long, link)

subjects

In [ ]:
subjects["MATH"]

这就是我们想要的。
顺便说一句:我们本可以采取不同的方法。注意，URL有一个与主题匹配的确定性查询参数
```
class_list.html?subject=MATH
```

如果只有主题短名称，我们还可以使用它来检索链接. 

#### 获取班级列表

接下来获取课程，让我们看看网站[计算机科学](https://student.apps.utah.edu/uofu/stu/ClassSchedules/main/1184/class_list.html?subject=CS).

我们将在传递课程名的函数中获取这个班级列表:

In [ ]:
def getWebsiteAsSoup(url):
    """ 
    Retrieve a website and return it as a BeautifulSoup object.   
    """
    
    #user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.167 Safari/537.36'
    #headers = {'User-Agent': user_agent}
      
    # values = {'name': 'Michael Foord',
    #       'location': 'Northampton',
    #       'language': 'Python' }
    
    #data = urllib.parse.urlencode(values)
    #data = data.encode('ascii')

    #req = urllib.request.Request(url, data, headers)
    req = urllib.request.Request(url)
    with urllib.request.urlopen(req) as response:
        classlist_html = response.read()
    
    print(classlist_html)
    
    class_soup = BeautifulSoup(classlist_html, 'html.parser')
    with open('class_list.html', 'w') as new_file:
        new_file.write(str(class_soup))
        
    return class_soup        

运行函数，查看输出:

In [ ]:
class_soup = getWebsiteAsSoup(subjects["CS"][1])
#print(class_soup[0:30000])

不幸的是，这个网站返回的HTML不是有效的HTML。看看这个部分: 

```html
<td colspan="15">
<p>
<span>Sections 2 - 6 belong to this lecture. This course requires registration for a lab section. Students will be automatically registered for this lecture section when registering for the pertinent lab section. </span>
</p>
<ul>
</ul>
</td></tr></table></div></div></div></div></main></section></body></html>


<tr class="even">
<td><a name="7136"></a><span>7136</span></td>
```

在这里，表、正文和html标记在html文档的中间结束。这可能是一个错误的网站代码，是动态生成的。但是，这不会出现在您的浏览器DOM中，所以很可能web服务器为scraper (python用户代理)生成的代码与为浏览器生成的代码不同。
我们有三种选择: 

 1. 用urllib找出返回正确HTML的头信息
 2. 使用自动化的浏览器模拟工具，如[selenium](http://www.seleniumhq.org/)
 3. 手工修改html中的错误，并且重新装载. 
 
我尝试了选项1(请参阅上面的注释代码，但是我找不到一个用户代理和浏览器配置为我提供有效的响应，所以我选择(非常粗糙的)选项3，在编辑器中手工修复返回的HTML，然后重新加载它: 

In [ ]:
class_soup = BeautifulSoup(open("class_list_modified.html"), "html.parser")

数据存储在一个表中，他有ID`classDetailsTable`. 让我们抽取表数据:

In [ ]:
# 现在我们来看一个例子，看看什么是好的特征
class_table = class_soup.find(id="classDetailsTable")
print(class_table)


当我们有一个表时，有一个简单的方法来获得数据——使用panda数据导入函数的魔力。有一个复杂的函数 [`read_html()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_html.html) 。 

在后台，它使用html5lib，这应该是anaconda安装的一部分，但如果不是，您将不得不安装它:

```
conda install -c anaconda html5lib
```

安装之后，我们可以将表作为字符串传递:

In [ ]:
import pandas as pd

classes = pd.read_html(str(class_table))[0]
classes.head(50)

这是一个表，但它看起来像脏数据的定义。还有一些多余的行我们要处理掉。看起来正确的行在"Sec"中有一个定义的值，我们会过滤掉没有值的行.

In [ ]:
mask = pd.notnull(classes["Sec."])
classes[mask]

这看起来更好。但是像面向对象编程一样，每个部分都有许多项。我们将过滤只包含讲座，研讨会或专题讲座. 

In [ ]:
classes = classes[(classes["Component"] == "Lecture") | (classes["Component"] == "Seminar") | (classes["Component"] == "Special Topics")]
classes

这学期谁教的讲座/研讨会/专题课程最多?

In [ ]:
classes["Instructor"].value_counts()

这个数据集还不是完全干净的。有多个不同版本的课程没有合并，而且共同教学的课程没有正确地分开。所以为了进行真正的分析，我们需要做更多的清理工作. 

### 计数注册人数

但是让我们看看哪个班有最多的学生。要做到这一点，我们必须后退几步，因为pandas的导入者没有保存网址，注册号码就隐藏在下一个网站后面。这里有一个例子 [data science lecture](https://student.apps.utah.edu/uofu/stu/ClassSchedules/main/1168/sections.html?subj=CS&catno=5963).

让我从这个网址抓取学生数量: 

In [ ]:
ds_enrollment_url = "https://student.apps.utah.edu/uofu/stu/ClassSchedules/main/1184/sections.html?subj=COMP&catno=5360"

with urllib.request.urlopen(ds_enrollment_url) as response:
    class_student_html = response.read()
class_student_soup = BeautifulSoup(class_student_html, 'html.parser')
# 使用pandas来读取table
student_ar = pd.read_html(str(class_student_soup))
student_ar

这个dataframe打包到一个数组中，我们可以从这个数组中得到dataframe:

In [ ]:
student_df = student_ar[0]
student_df

现在，我们可以获取学生数目:

In [ ]:
students = student_df["Currently Enrolled"][0]
students

让我们将所有这些打包到一个函数中，该函数接受URL并返回数字:

In [ ]:
def scrape_students(url):
    """
    从URL中指定的网站表中检索学生人数
    """
    with urllib.request.urlopen(url) as response:
        class_student_html = response.read()
    class_student_soup = BeautifulSoup(class_student_html, 'html.parser')
    # 使用pandas来读取table
    student_df = pd.read_html(str(class_student_soup))[0]
    students = student_df["Currently Enrolled"][0]
    return students

In [ ]:
# test it with the datascience lecture
scrape_students(ds_enrollment_url)    

现在我们仍然需要提取URL来传递到这个函数和其他我们关心的网站属性中。在这里，我们将创建一个量身定制的解决方案的网站-这当然是脆弱的，并将打破，如果网站的变化(一个网站的DOM不应该是一个稳定的API)。但这是我们能做的最好的了。
我们将创建一个矩阵，用于初始化一个dataframe。记住，我们已经将表抽取到' class_table '中了。

In [ ]:
print(class_table)

通常，这不是很好读。这是检查器版本。

我们看到没有内容的行有两个属性:它们有一个类属性' notes '，而只有一个嵌套的' <td> '—与包含数据的行形成对比。我们可以使用这些属性来选择正确的数据集。

In [ ]:
# 这些是我们关心的列columns
header = ["Subject", "Number", "Section", "Type", "Title", "Instructor", "Students" ]
matrix = []

# 针对每个表的行
for row in class_table.find_all("tr"):
    # 抽取tds
    tds = row.find_all("td")
   
    # 表中的列是 
    # Class; Subject; Cat. #; Sec.; Component; Units; Title; Days/Time & Session; 
    # Location; Class Attrs; Instructor; Feed back; Pre Req; Fees
    # 我们想要定义在header中的数据
    # 如果只有单个td则跳过这行，因为他是“notes”行的其中之一.
    if (len(tds) > 1):
        # print(tds)
        record = []
        # 这里，我们手工抽取想要的字段。很丑，但有效. 
        record.append(tds[1].text.strip()) # subject
        record.append(tds[2].text.strip()) # cat #
        record.append(tds[3].text.strip()) # Section
        record.append(tds[4].text.strip()) # Component/Type
        record.append(tds[6].text.strip()) # Title
        record.append(tds[10].text.strip()) # Instructor
        # 重构URL:
        student_url = url+tds[2].find("a").get("href")
        # 这里我们称之为scrape_students，它提取学生的数量  
        students = scrape_students(student_url)
        record.append(students)
        # print(record)
        matrix.append(record)

让我们看看矩阵中的第一个元素:

In [ ]:
matrix[0:4]

看起来不错，把它放到dataframe中:

In [ ]:
classes = pd.DataFrame(matrix, columns=header)

退回到之前我们做的清理操作:

In [ ]:
classes = classes[(classes["Type"] == "Lecture") | (classes["Type"] == "Seminar") | (classes["Type"] == "Special Topics")]
classes.head()

当我们有分组的时候，学生通常被登记在主要的课程“001”除了登记在组(“002”，“003”等)。让我们只计算主要的部分。这并不是对所有情况都有效，但是自动修复这个问题是困难的。

In [ ]:
mask = classes["Section"] == "001"
classes = classes[mask]
classes.head()

让我们看看哪个是最大的班级:

In [ ]:
classes = classes.sort_values("Students", ascending=False)
classes = classes.reset_index(drop=True)
classes.head()

现在，我们想要回答这个问题 **哪个级别level的CS学生最多?** 级别level是课程编号的前位数。让我们创建一个新的级别并将其添加到dataframe: 

In [ ]:
classes["level"] = classes["Number"].map(lambda x: (str(x)[0]))
classes

可以用groupby计算出答案:

In [ ]:
import numpy as np
student_stats = classes.groupby("level").aggregate([np.sum, np.mean, np.std])
student_stats

使用箱图进行可视化:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
props = dict(linewidth=2)
# here we group by level
classes.boxplot(by='level', figsize=(10, 10),  boxprops=props, medianprops=props, whiskerprops=props, capprops=props)

使用分离的图形方法:

In [ ]:
classes.groupby("level").boxplot(figsize=(13, 13), boxprops=props, medianprops=props, whiskerprops=props, capprops=props)

然后画一个条形图表示班上学生的人数:

In [ ]:
student_sum = classes.groupby("level").aggregate([np.sum])
student_sum.plot.bar(legend=False, figsize=(14, 8))

我们可以看到，大多数学生在3000级别，而另一个高峰是在6000级别的班级.

## 四、网页JSON数据爬取与解析

 我们将使用主要的API形式REST(具象状态传输，REpresentational State Transfer)，它现在是在web上公开和API的主要方式。REST api使用[uri](a Uniform Resource Identifier统一资源标识符; URLs是 URIs的特定形式)来指定我们要处理的API.
 REST api可以以不同的形式返回数据，最常见的是JSON和XML，在这两者中，JSON现在占主导地位。
JSON代表JavaScript对象表示法，是一种非常方便的格式.下面我们使用request库来获取国际空间站ISS的当前位置

In [ ]:
import requests 
# 通过opennotify api请求获取国际空间站ISS的最新位置.
response = requests.get("http://api.open-notify.org/iss-now.json")

response

看起来不错，我们收到了回复，状态是200。200是什么意思?这是一个状态码——你可能在网上看到过“404错误”。
以下是一些代码:

** *200** -一切正常，结果已经返回(如果有的话)

** *301** -服务器将您重定向到另一个端点。当公司切换域名或端点名称更改时，可能会发生这种情况。

** *401** -服务器认为您没有经过身份验证。当您没有发送正确的凭证来访问API时，就会发生这种情况(稍后我们将讨论身份验证)。

** *400** -服务器认为你做了一个错误的请求。这种情况可能发生在您没有发送正确的数据时。

** *403** -您试图访问的资源是被禁止的-您没有正确的权限来查看它。

** *404** -您试图访问的资源在服务器上没有找到。

让我们尝试得到一个错误的响应URL:

In [ ]:
response_failed = requests.get("http://api.open-notify.org/iss")

response_failed

In [ ]:
response.content

In [ ]:
response.headers['content-type']

In [ ]:
response_j = response.content.decode("utf-8")
print(response_j)

让我们来看看JSON:

```JSON
{
  "iss_position": {
    "latitude": -30.005751854107206, 
    "longitude": -104.20085371352678
  }, 
  "message": "success", 
  "timestamp": 1475240215
}
```

看起来像个字典，有key-value对. 

可以使用[json library](https://docs.python.org/3/library/json.html)将JSON转换为对象:

In [ ]:
import json
response_d = json.loads(response_j)
print(type(response_d))
print(response_d)
response_d["iss_position"]

In [ ]:
#pandas也可以读入json
import pandas as pd 

df = pd.read_json(response_j)
df

## 五、对于HTML的Table标签组织的表格数据，还可以是使用Pandas直接读取
如：要读取的网页表格数据
http://vip.stock.finance.sina.com.cn/q/go.php/vComStockHold/kind/jjzc/index.phtml

In [3]:
import pandas as pd

# 数据出现省略号
pd.set_option('display.width', None)

url = 'http://vip.stock.finance.sina.com.cn/q/go.php/vComStockHold/kind/jjzc/index.phtml'

# 可能有多个表格，我们取第一个
df = pd.read_html(url)[0]
df.head()


,代码,简称,截至日期,家数,本期持股数(万股),持股占已流通A股比例(%),同上期增减(万股),持股比例(%),上期家数,明细
0,2007,华兰生物,2020-03-31,3,6049.2045,4.31,-1907.0929,5.67,3,+展开明细
1,2913,奥士康,2020-03-31,1,159.9550,1.08,45.2351,0.78,3,+展开明细


## 六、爬取总结

抓取是一种从网站获取信息的方式，而网站的设计并不是为了让数据可访问。因此，它经常是“脆弱的”:网站的改变会破坏你的抓取脚本。它通常也不受欢迎，因为scaper会导致很多流量. 

我们在这里提取信息的方式也假定了HTML是基于URL一致地生成的。不幸的是，随着网站适应浏览器类型、分辨率、地区设置，以及大量内容被动态加载(如通过web-sockets)，这种情况越来越少见了。例如，许多网站现在自动加载更多的数据，一旦你滚动到页面底部。这些网站不能用我们的方法，而是一个浏览器仿真方法，使用例如[Selenium]() 会是必须的. [这是向导](https://medium.com/the-andela-way/introduction-to-web-scraping-using-selenium-7ec377a8cf72) . 

最后，许多服务通过定义良好的接口(API)来提供它们的数据。使用API总是比抓取更好的主意，但是抓取是一个很好的后备方法!

## 练习-1--urllib的基本应用

In [ ]:
#读取并显示网页内容

import urllib.request
fp = urllib.request.urlopen(r'http://www.jd.com')
print(fp.read(100))              #读取100个字节
print(fp.read(100))     #使用UTF8进行解码
fp.close()                       #关闭连接


In [ ]:
#使用GET方法读取并显示指定url的内容
"""
GET把参数包含在URL中，POST通过request body传递参数
GET在浏览器回退时是无害的，而POST会再次提交请求。
 
GET产生的URL地址可以被Bookmark，而POST不可以。
 
GET请求会被浏览器主动cache，而POST不会，除非手动设置。
 
GET请求只能进行url编码，而POST支持多种编码方式。
 
GET请求参数会被完整保留在浏览器历史记录里，而POST中的参数不会被保留。
 
GET请求在URL中传送的参数是有长度限制的，而POST没有。
 
对参数的数据类型，GET只接受ASCII字符，而POST没有限制。
 
GET比POST更不安全，因为参数直接暴露在URL上，所以不能用来传递敏感信息。
 
GET参数通过URL传递，POST放在Request body中。
"""
import urllib.request
import urllib.parse
#params = urllib.parse.urlencode({'spam': 1, 'eggs': 2, 'bacon': 0})
#url = "http://www.musi-cal.com/cgi-bin/query?%s" % params
url = "http://tianqihoubao.com/weather/province.aspx?id=340000"

with urllib.request.urlopen(url) as f:
    print(f.read(100))


In [ ]:
#使用POST方法读取并显示指定url的内容
import urllib.request
import urllib.parse
data = urllib.parse.urlencode({'spam': 1, 'eggs': 2, 'bacon': 0})
data = data.encode('ascii')
with urllib.request.urlopen("http://requestb.in/xrbl82xr", data) as f:
    print(f.read().decode('utf-8'))


In [ ]:
"""
爬取公众号文章中的图片.
第1步  确定公众号文章的地址，以微信公众号“Python小屋”里的一篇文章为例，文章标题为“报告PPT（163页）：基于Python语言的课程群建设探讨与实践”，
第2步  在浏览器（以Chrome为例）中打开该文章，然后单击鼠标右键，选择“查看网页源代码”，分析后发现，公众号文章中的图片链接格式为：

<p><img data-s="300,640" data-type="png" data-src="http://mmbiz.qpic.cn/mmbiz_png/xXrickrc6JTO9TThicnuGGR7DtzWtslaBlYS5QJ73u2WpzPW8KX8iaCdWcNYia5YjYpx89K78YwrDamtkxmUXuXJfA/0?wx_fmt=png" style="" class="" data-ratio="0.5580865603644647" data-w="878"  /></p>
第3步  根据前面的分析，确定用来提取文章中图片链接的正则表达式：

pattern = 'data-type="png" data-src="(.+?)"'

第4步  编写并运行Python爬虫程序，代码如下

"""
from re import findall
from urllib.request import urlopen

url = 'https://mp.weixin.qq.com/s?__biz=MzI4MzM2MDgyMQ==&mid=2247486249&idx=1&sn=a37d079f541b194970428fb2fd7a1ed4&chksm=eb8aa073dcfd2965f2d48c5ae9341a7f8a1c2ae2c79a68c7d2476d8573c91e1de2e237c98534&scene=21#wechat_redirect'
with urlopen(url) as fp:
    content = fp.read().decode()

pattern = 'data-type="png" data-src="(.+?)"'
#查找所有图片链接地址
result = findall(pattern, content)
#逐个读取图片数据，并写入本地文件
for index, item in enumerate(result):
    with urlopen(str(item)) as fp:
        with open(str(index)+'.png', 'wb') as fp1:
            fp1.write(fp.read())


## 练习-2--requests的基本应用

In [ ]:
#读取并下载指定的URL的图片文件
import requests
picUrl = r'https://www.python.org/static/opengraph-icon-200x200.png'
r = requests.get(picUrl)
r.status_code
with open('pic.png', 'wb') as fp:
    fp.write(r.content)                #把图像数据写入本地文件


## 练习-3--selenium爬虫案例

#使用selenium编写爬虫程序，获取指定城市的当前天气

"""
第1步  首先，查看一下本地计算机Windows操作系统的内部版本号，以我的Win10为例，步骤为：依次单击开始==>设置==>系统==>关于，找到下图中的操作系统内部版本号

第2步  打开网址https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/，下载合适版本的驱动，并放到Python安装目录下。

第3步  打开命令提示符环境，使用pip安装扩展库selenium。

第4步  编写如下程序代码

"""

In [1]:
#注意：对于动态网页，可能不会正常获取JavaScript动态生成的内容，从而导致解析失败，必要时可以使用driver.execute_script方法执行JS脚本
#使用python调用selenium框架执行JavaScript脚本也可以将JavaScript的执行结果返回给python的一个对象，对象类型是WebElement，只需要在调用的JavaScript脚本中使用return 语句返回对应的内容即可
#
import re
from selenium import webdriver

#指定引擎
driver = webdriver.Edge()
city = input('请输入要查询的城市：').lower()
#获取指定URL的信息，并进行渲染
driver.get(r'http://openweathermap.org/find?q={0}'.format(city))
#网页内容渲染结束之后获取网页源代码，并转换成小写
content = driver.page_source.lower()
matchResult = re.search(r'<a href="(.+?)">\s+'+city+'.+?]', content)
if matchResult:
    print(matchResult.group(0))
else:
    print('查不到，请检查城市名字。')
# 关闭当前页面，如果只有一个页面，会关闭浏览器
# driver.close()

# 关闭浏览器
driver.quit()

请输入要查询的城市：Beijing
<a href="/city/1816670"> beijing, cn</a></b> <img src="http://openweathermap.org/images/flags/cn.png"><b><i> clear sky</i></b><p><span class="badge badge-info">10.5°с </span> temperature from 8.3 to 14.4 °с, wind 3 m/s. clouds 0 %, 1021 hpa</p><p>geo coords <a href="/weathermap?zoom=12&amp;lat=39.9075&amp;lon=116.3972">[39.9075, 116.3972]


In [2]:
content

'<html lang="en"><head>\n        <meta http-equiv="content-type" content="text/html; charset=utf-8">    \n    <meta http-equiv="expires" content="tue, sep 20 2018 15:27:22 gmt">\n    <meta http-equiv="last-modified" content="tue, sep 20 2018 15:27:22 gmt">\n    <!--<meta http-equiv="x-ua-compatible" content="ie=edge">-->\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <meta name="keywords" content="weather, world, openstreetmap, forecast, openweather, current">\n    <meta name="description" content="">\n    <meta name="author" content="openweathermap.org">\n    <meta name="domain" content="openweathermap.org">        \n    <meta name="p:domain_verify" content="34fe229eab8562dca90f4a5962ff80a9">\n    <meta content="find" property="title">            \n        <link href="/themes/openweathermap/assets/vendor/owm/img/icons/favicon.ico" rel="shortcut icon">\n<link href="/themes/openweathermap/assets/vendor/owm/img/icons/logo_16x16.png" rel="icon" type="imag

In [5]:
#如果返回的内容与正则表达式不匹配，可能是返回内容没有获取完整或错误所致，可以用以下代码测试：
#读取并下载指定的URL的图片文件
import re
import requests
city = input('请输入要查询的城市：').lower()
weather_url=r'http://openweathermap.org/find?q={0}'.format(city)
print("weather_url:",weather_url)
r = requests.get(weather_url)
print(r.status_code)
content=r.content.decode("UTF-8").lower()
print(content)
matchResult = re.search(r'<a href="(.+?)">\s+'+city+'.+?]', content)
if matchResult:
    print(matchResult.group(0))
else:
    print('查不到，请检查城市名字。')

#从打印网页返回内容可以看出，返回的网页中没有获取JavaScript的动态数据内容，而JavaScript代码所对应的变量值，
#如name（对应beijing）、tmin（最小温度）、tmax（最大温度）、temp（温度）等都没有正常获取和替换网页对应内容部分

请输入要查询的城市：beijing
weather_url: http://openweathermap.org/find?q=beijing
200
<!doctype html>
<html lang='en'>
    <head>
        <meta http-equiv="content-type" content="text/html; charset=utf-8">    
    <meta  http-equiv=expires content="tue, sep 20 2018 15:27:22 gmt">
    <meta http-equiv="last-modified" content="tue, sep 20 2018 15:27:22 gmt">
    <!--<meta http-equiv="x-ua-compatible" content="ie=edge">-->
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta name="keywords" content="weather, world, openstreetmap, forecast, openweather, current" >
    <meta name="description" content="" >
    <meta name="author" content="openweathermap.org">
    <meta name="domain" content="openweathermap.org" >        
    <meta name="p:domain_verify" content="34fe229eab8562dca90f4a5962ff80a9"/>
    <meta property="title" content="find" />            
        <link rel="shortcut icon" href="/themes/openweathermap/assets/vendor/owm/img/icons/favicon.ico" />
<link rel=

In [ ]:
# PhantomJS不需要启动浏览器，但是可以模拟浏览器的操作，效率较高。
#但是在Python3的后续版本中，可能在selenium模块中移除对PhantomJS的支持。可以选择使用headless版本的chrome或Firefox驱动


# PhontamJS 测试代码

# 导入 webdriver
from selenium import webdriver
import re
# 要想调用键盘按键操作需要引入keys包
from selenium.webdriver.common.keys import Keys

# 调用环境变量指定的PhantomJS浏览器创建浏览器对象
driver = webdriver.PhantomJS()

# 如果没有在环境变量指定PhantomJS位置
# driver = webdriver.PhantomJS(executable_path="./phantomjs-2.1.1"))

# get方法会一直等到页面被完全加载，然后才会继续程序，通常测试会在这里选择 time.sleep(2)
city = input('请输入要查询的城市：').lower()
#获取指定URL的信息，并进行渲染
driver.get(r'http://openweathermap.org/find?q={0}'.format(city))
#网页内容渲染结束之后获取网页源代码，并转换成小写
content = driver.page_source.lower()
print("Web Content:",content)
matchResult = re.search(r'<a href="(.+?)">\s+'+city+'.+?]', content)
if matchResult:
    print(matchResult.group(0))
else:
    print('查不到，请检查城市名字。')
# 获取当前url
print(driver.current_url)
# 关闭当前页面，如果只有一个页面，会关闭浏览器
# driver.close()

# 关闭浏览器
driver.quit()

In [3]:
#新版本的Selenium不再支持PhantomJS了，使用Chrome或Firefox的无头版本来替代
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')

#指定本地下载的与浏览器版本一致的驱动程序路径，不然会使用远程服务器缺省版本，导致版本不一致出错
#如将下载的驱动拷贝到D:/Anaconda3目录下，指定路径时需要包括路径和驱动程序文件名（不含扩展名）
#chrome驱动程序下载地址：http://npm.taobao.org/mirrors/chromedriver/；或 http://chromedriver.storage.googleapis.com/index.html
path='D:/Anaconda3/chromedriver'
#driver = webdriver.Chrome(executable_path=path, options=chrome_options)
driver = webdriver.Chrome(path)
#driver = webdriver.Chrome(executable_path=path, options=chrome_options)
#可能出错，This version of ChromeDriver only supports Chrome version 75
#driver = webdriver.Chrome(options=chrome_options) 
# get方法会一直等到页面被完全加载，然后才会继续程序，通常测试会在这里选择 time.sleep(2)
city = input('请输入要查询的城市：').lower()
#获取指定URL的信息，并进行渲染
driver.get(r'http://openweathermap.org/find?q={0}'.format(city))
#网页内容渲染结束之后获取网页源代码，并转换成小写
content = driver.page_source.lower()
#print("Web Content:",content)
matchResult = re.search(r'<a href="(.+?)">\s+'+city+'.+?]', content)
if matchResult:
    print(matchResult.group(0))
else:
    print('查不到，请检查城市名字。')
# 获取当前url
print(driver.current_url)
# 关闭当前页面，如果只有一个页面，会关闭浏览器
driver.close()

# 关闭浏览器
driver.quit()

请输入要查询的城市：Beijing
<a href="/city/1816670"> beijing, cn</a></b> <img src="http://openweathermap.org/images/flags/cn.png"><b><i> clear sky</i></b><p><span class="badge badge-info">6.7°с </span> temperature from 6 to 7.8 °с, wind 1 m/s. clouds 0 %, 1022 hpa</p><p>geo coords <a href="/weathermap?zoom=12&amp;lat=39.9075&amp;lon=116.3972">[39.9075, 116.3972]
https://openweathermap.org/find?q=beijing


## 练习-4--爬取彼岸桌面壁纸图片

爬取 [彼岸网站壁纸图片](http://www.netbian.com/weimei/index.htm) . 

In [1]:
import requests
import os
import re
import time


def Picture_Download(url_img_path, img_title):  # 定义一个图片下载函数，传入图片Url地址,图片标题进行下载保存
    file_name = img_title.replace('/', ' ').strip()  # 因为保存文件名中不能有/，所以要把/替换成空，不然会保存错误
    try:
        result = requests.get(url_img_path.strip())  # 使用GET方法请求图片Url地址
    except:
        print(url_img_path, ' Download failed')
    else:
        if result.status_code == 200:  # 如果响应状态码为200，说明文件存在，则保存图片
            File = open(file_name + '.jpg', 'wb')
            File.write(result.content)
            File.close()


def Img_Url(url):  # 通过传入网址读取该页面的所有壁纸图片地址及壁纸标题
    result = requests.get(url)
    result.encoding = 'gbk'  # 该网页编码为gbk编码
    compile = re.compile(r'<img src="(.*?)" alt="(.*?)" />')  # 使用正则表达式提取壁纸地址及壁纸标题
    all = compile.findall(result.text)
    for item in all:
        print(item[0], item[1])
        Picture_Download(item[0], item[1])  # 循环取出一页中的每一个壁纸地址及壁纸标题，传入Picture_Download函数进行下载保存


def main():
    #for i in range(1, 74):  # 总共有73页，故要进行1-74循环
    for i in range(1, 3):    
        if i == 1:
            Img_Url(r'http://www.netbian.com/weimei/index.htm')
        else:
            Img_Url(r'http://www.netbian.com/weimei/index_%d.htm' % i)
            time.sleep(2)  # 提取一页就暂停2秒，防止访问过快被屏蔽


if __name__ == '__main__':
    main()

http://img.netbian.com/file/newc/e4f018f89fe9f825753866abafee383f.jpg 星空 女孩 观望 唯美夜景壁纸
http://img.netbian.com/file/2019/1206/smalldc5b87229ac834db90b35b41785fe2df1575600237.jpg 春天山脉小河流树和花2k唯美风景壁纸
http://img.netbian.com/file/2020/0107/63562ba62a7cd23bea9992db97e07095.jpg 4K/5K/8K高清壁纸
http://img.netbian.com/file/2019/1206/small4bb3e43f66b5a919fea1a9005be11b861575600181.jpg 冬天小矮人和山脉唯美壁纸
http://img.netbian.com/file/2019/1114/small56ce022d46a49e1feefec0a068b4fd1a1573698270.jpg 唯美大海 树 鱼 月亮 唯美意境壁纸
http://img.netbian.com/file/2019/0529/smallc7f3778a6fd8bfb157d3cb6ada90646a1559095022.jpg 夜舞唯美2k壁纸
http://img.netbian.com/file/2019/0317/small096666d63b4374e65b33093f4ed253d11552831886.jpg 狼 森林 山 瀑布 唯美插画壁纸
http://img.netbian.com/file/2019/0312/small84b4b5a8ba3e60871464cfdaa261da551552405061.jpg 树林的少女和精灵唯美插画壁纸
http://img.netbian.com/file/2019/0212/small2777917c13ea37c90615505973456d881549939165.jpg 热爱大自然的人壁纸
http://img.netbian.com/file/2019/0212/small8244baf212e210ce647ec4f2cc1e5faa1549938695.jpg 鱼 荷花

In [ ]:
#简短版solution,无函数封装，只爬取单页
from re import findall
from urllib.request import urlopen

url=r'http://www.netbian.com/weimei/index.htm'
with urlopen(url) as fp:
    content=fp.read().decode("gb2312")
#print(content)
pattern=r'<img src="(.*?)" alt="(.*?)" />'
#pattern='^target="_blank"><img src="'
result=findall(pattern,content)
#注意匹配搜索到的数据结构是个列表，元素是是个包含url和图片名称的元组
#如：('http://img.netbian.com/file/newc/11d3e21f332088d548610495aab98b8d.jpg', '三匹马唯美壁纸')
print(result)  
for item in result:
    #item[0]对应URL地址，item[1]对应图片名称
    print(item[0], item[1])
    file_name = item[1].replace('/', ' ').strip()  # 因为保存文件名中不能有/，所以要把/替换成空，不然会保存错误
    with urlopen(item[0]) as fp1:
        with open(file_name+'.png',"wb") as fp2:
            fp2.write(fp1.read())

## 练习-5--爬取酷狗音乐列表

爬取 [酷狗音乐网站](http://www.kugou.com/yy/rank/home)的音乐列表 . 

In [3]:
import requests
from bs4 import BeautifulSoup
import time

headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
}

def get_info(url):
    wb_data = requests.get(url,headers=headers)
    soup = BeautifulSoup(wb_data.text,'lxml')
    print(soup)
    ranks = soup.select('span.pc_temp_num')
    titles = soup.select('div.pc_temp_songlist > ul > li > a')
    times = soup.select('span.pc_temp_tips_r > span')
    for rank,title,time in zip(ranks,titles,times):
        data = {
            'rank':rank.get_text().strip(),
            'singer':title.get_text().split('-')[0],
            'song':title.get_text().split('-')[1],
            'time':time.get_text().strip()
        }
        print(data)

if __name__ == '__main__':
    #urls = ['http://www.kugou.com/yy/rank/home/{}-8888.html'.format(str(i)) for i in range(1,24)]
    urls = ['http://www.kugou.com/yy/rank/home/{}-8888.html'.format(str(i)) for i in range(1,2)]
    for url in urls:
        get_info(url)
        time.sleep(1)



<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="酷狗2017正式版, 歌手,明星,音乐,在线音乐,在线听歌,听歌,新专辑,港台,日本,韩国,欧美,英国" name="keywords"/>
<meta content="酷狗官方网站是中国最新最全的在线正版音乐网站,提供最新的在线音乐服务、免费音乐下载、最新的音乐播放器下载。" name="description"/>
<link href="//static.kgimg.com/" rel="dns-prefetch"/>
<link href="//sdn.kugou.com/" rel="dns-prefetch"/>
<link href="//js.webcollect.kugou.com/" rel="dns-prefetch"/>
<title>酷狗TOP500_排行榜_乐库频道_酷狗网</title>
<script data-embed="false" src="https://www.kugou.com/yy/static/js/PCToMoblie.js" type="text/javascript"></script>
<link href="https://www.kugou.com/yy/static/images/favicon.ico" rel="shortcut icon"/>
<link href="https://www.kugou.com/yy/static/css/rankPage.min.css?201505211743" rel="stylesheet" type="text/css"/>
</head>
<body>
<link data-embed="" href="https://www.